# Server Test
> This is the code I am using to test the server. In this test, I will use 2 connections and attempt to make the join the same game.

In [213]:
import socket as s

## Connecting to server

I connect to the GO server on port 80(configured in docker).

In [214]:
conn = s.socket(s.AF_INET, s.SOCK_STREAM)
conn.connect(("127.0.0.1", 80))

## Sending packets

All packets must be suffixed with \n. This is a python implemtation of how to send them:

In [215]:
# Sends a packet, returns whatever socket.Send returns
def send_packet(content: bytes):
    global conn
    return conn.send(content + b'\n')


## Creating a game

I need to send a packet with 'c' as the first character to intialise a game:

In [216]:
send_packet(b'c')

2

In response, I should get a packet with the *game code*.

In [217]:
BUFFER_SIZE = 1024

def recieve_packet() -> bytes:
    return conn.recv(BUFFER_SIZE)

In [218]:
packet = recieve_packet()

In [219]:
trimmed = packet[1:]
import json
packetDict = json.loads(trimmed)

## User 2

I will attempt to join the game as another user

In [220]:
conn2 = s.socket(s.AF_INET, s.SOCK_STREAM)
conn2.connect(("127.0.0.1", 80))

In [221]:
conn2.send(b'j{"Name":"' + packetDict['Name'].encode() + b'"}\n')
conn2.recv(BUFFER_SIZE)

b'Jsuccess\n'


## Playing the game
The two players should be able to share data with each other.

In [222]:
conn.send(
    b'f{"X":1, "Y":1, "Dx":999, "Dy":999}\n'
)

36

In [223]:
conn.recv(BUFFER_SIZE)

b'F{"Players":[{"Name":"wjlwpxyfld","X":1,"Y":1,"Dx":999,"Dy":999},{"Name":"mwdvnwcdmj","X":0,"Y":0,"Dx":0,"Dy":0}],"State":1,"Name":"oianydbk"}\n'

subequent reads should have different values for player 1, becuase Dx and Dy move the player along like so

In [224]:
import time
time.sleep(1)

conn2.send(
    b'f{"X":1, "Y":1, "Dx":0, "Dy":0}\n'
)
conn2.recv(BUFFER_SIZE)

b'F{"Players":[{"Name":"wjlwpxyfld","X":1,"Y":1,"Dx":999,"Dy":999},{"Name":"mwdvnwcdmj","X":1,"Y":1,"Dx":0,"Dy":0}],"State":1,"Name":"oianydbk"}\n'

## Error cases

In [225]:
conn3 = s.socket(s.AF_INET, s.SOCK_STREAM)
conn3.connect(("127.0.0.1", 80))

### Joining full/running game
This should fail and return an error packet explaining why

In [226]:
conn3.send(b'j{"Name":"' + packetDict['Name'].encode() + b'"}\n')
conn3.recv(BUFFER_SIZE)

b'Ecould not join game\n'

### Creating a game with 1 player and attempting to send a frame
This should fail and return an error packet explaining why

In [227]:
conn3.send(b'c\n')
conn3.recv(BUFFER_SIZE)#i dont care what the game id is

conn3.send(
    b'f{"X":1, "Y":1, "Dx":0, "Dy":0}\n'
)
conn3.recv(BUFFER_SIZE)

b'EGame not started yet\n'

# Logs

> Here are the logs on the server from one run if you're interested.

> I restarted the notebook so it actually ran twice to see how the server handles creating and destroying connections, I had to implement mutexes to stop the server crashing. You can see where connections were closed becuase of the EOF errors, which trigger the server to abandon the connection.

> It's not very ordered becuase this is a multithreaded server meaning:
>   - Computers are already pretty fast so there is not much difference in times
>   - This specific computer is managing up to 3 connections simultaneously
>
> This produced the mess you see below becuase only one thread can write to stdout at a time

    2023-08-18 19:13:47 Press enter to stop server
    2023-08-18 19:13:52 New player connection from 172.17.0.1:43980 
    2023-08-18 19:13:52 Player fbwfpvejac attempting to host game
    2023-08-18 19:13:52 Player fbwfpvejac leaving game
    2023-08-18 19:13:52 Player fbwfpvejac is having thier position reset
    2023-08-18 19:13:52 New player connection from 172.17.0.1:43998 
    2023-08-18 19:13:52 Player qzpsvmiryr joining game mlgnopdk
    2023-08-18 19:13:52 Player qzpsvmiryr leaving game
    2023-08-18 19:13:52 Player qzpsvmiryr is having thier position reset
    2023-08-18 19:13:52 Player fbwfpvejac applying frame
    2023-08-18 19:13:53 Player qzpsvmiryr applying frame
    2023-08-18 19:13:53 New player connection from 172.17.0.1:44024 
    2023-08-18 19:13:54 Player mpxtqjfzkx joining game mlgnopdk
    2023-08-18 19:13:54 Player mpxtqjfzkx leaving game
    2023-08-18 19:13:54 Player mpxtqjfzkx attempting to host game
    2023-08-18 19:13:54 Player mpxtqjfzkx leaving game
    2023-08-18 19:13:54 Player mpxtqjfzkx is having thier position reset
    2023-08-18 19:13:54 Player mpxtqjfzkx applying frame
    2023-08-18 19:14:39 Error while recieving fron conn: EOF, closing... 
    2023-08-18 19:14:39 Conn closed
    2023-08-18 19:14:39 Player fbwfpvejac is being disposed
    2023-08-18 19:14:39 Player fbwfpvejac leaving game
    2023-08-18 19:14:39 New player connection from 172.17.0.1:35708 
    2023-08-18 19:14:39 Player yogwcqyzzl attempting to host game
    2023-08-18 19:14:39 Player yogwcqyzzl leaving game
    2023-08-18 19:14:39 Player yogwcqyzzl is having thier position reset
    2023-08-18 19:14:39 Error while recieving fron conn: EOF, closing... 
    2023-08-18 19:14:39 Conn closed
    2023-08-18 19:14:39 Player qzpsvmiryr is being disposed
    2023-08-18 19:14:39 Player qzpsvmiryr leaving game
    2023-08-18 19:14:39 New player connection from 172.17.0.1:35722 
    2023-08-18 19:14:39 Player fmpidfrwsz joining game ctxddnbz
    2023-08-18 19:14:39 Player fmpidfrwsz leaving game
    2023-08-18 19:14:39 Player fmpidfrwsz is having thier position reset
    2023-08-18 19:14:40 Player yogwcqyzzl applying frame
    2023-08-18 19:14:41 Player fmpidfrwsz applying frame
    2023-08-18 19:14:41 Error while recieving fron conn: EOF, closing... 
    2023-08-18 19:14:41 Conn closed
    2023-08-18 19:14:41 Player mpxtqjfzkx is being disposed
    2023-08-18 19:14:41 Player mpxtqjfzkx leaving game
    2023-08-18 19:14:41 New player connection from 172.17.0.1:35742 
    2023-08-18 19:14:41 Player ptvmfuduas joining game ctxddnbz
    2023-08-18 19:14:41 Player ptvmfuduas leaving game
    2023-08-18 19:14:41 Player ptvmfuduas attempting to host game
    2023-08-18 19:14:41 Player ptvmfuduas leaving game
    2023-08-18 19:14:41 Player ptvmfuduas is having thier position reset
    2023-08-18 19:14:41 Player ptvmfuduas applying frame
